In [21]:
%pip install requests python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [22]:
import requests
import urllib
import pandas as pd
from dotenv import load_dotenv
import logging
import os

load_dotenv()
API_TOKEN = os.getenv('API_TOKEN')
CLAN_NAME = os.getenv('CLAN_NAME')

In [23]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(message)s')

In [24]:
headers = { "Authorization": f'Bearer {API_TOKEN}'}

def getclan():
    return requests.get(f'https://api.clashroyale.com/v1/clans?name={CLAN_NAME}', headers = headers).json()["items"][0]

def getbattlelog(player_tag):
    player_tag = urllib.parse.quote(player_tag, safe='')
    data = requests.get(f'https://api.clashroyale.com/v1/players/{player_tag}/battlelog', headers = headers).json()

    team = data["team"]
    opponent = data["opponent"]
    
    return team,opponent,data

def getmembers(clan_tag):
    clan_tag = urllib.parse.quote(clan_tag, safe='')
    playerlist = requests.get(f'https://api.clashroyale.com/v1/clans/{clan_tag}/members', headers = headers).json()
    
    logging.info(f"Found {len(playerlist['items'])} members in clan {CLAN_NAME}")
    
    player_data = []

    for player in playerlist['items']:
        player_tag = urllib.parse.quote(player['tag'], safe='')
        player_data.append(requests.get(f'https://api.clashroyale.com/v1/players/{player_tag}', headers = headers).json())
        if player["name"] == "dogbag":
            print(player["tag"])

    return player_data

        
clan = getclan()
members=getmembers(clan["tag"])
df = pd.DataFrame(members)



INFO:Found 10 members in clan S.J.G.R


#V8LVP0028


In [25]:
df.columns

Index(['tag', 'name', 'expLevel', 'trophies', 'bestTrophies', 'wins', 'losses',
       'battleCount', 'threeCrownWins', 'challengeCardsWon',
       'challengeMaxWins', 'tournamentCardsWon', 'tournamentBattleCount',
       'role', 'donations', 'donationsReceived', 'totalDonations',
       'warDayWins', 'clanCardsCollected', 'clan', 'arena', 'leagueStatistics',
       'badges', 'achievements', 'cards', 'supportCards', 'currentDeck',
       'currentDeckSupportCards', 'currentFavouriteCard', 'starPoints',
       'expPoints', 'legacyTrophyRoadHighScore',
       'currentPathOfLegendSeasonResult', 'lastPathOfLegendSeasonResult',
       'bestPathOfLegendSeasonResult', 'progress', 'totalExpPoints'],
      dtype='object')

In [27]:
data, team, opponent = getbattlelog(members["items"][0]["tag"])

pd.DataFrame(team)

TypeError: list indices must be integers or slices, not str